# Image Feature Extraction

## Завдання

3.  Завантажте датасет **CIFAR-10** та використайте **HOG** для витягування ознак зображень. Застосуйте будь-який простий класифікатор (наприклад, **SVM**) для класифікації зображень та обчисліть метрики точності, такі як **accuracy, precision, recall та F1-score.

4.  Використайте метод **SIFT** для витягування ознак з зображень та порівняйте його з методом **HOG** за допомогою метрик точності.

## Розв'язок

In [47]:
import cv2
import numpy as np

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from skimage.feature import hog
from skimage import color
from tensorflow.keras.datasets import cifar10
from sklearn.decomposition import PCA
import numpy as np

### 3. Використання HOG для витягування ознак зображень

In [5]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [6]:
train_data_gray = np.array([cv2.cvtColor(x.reshape(32, 32, 3), cv2.COLOR_BGR2GRAY) for x in x_train])
test_data_gray = np.array([cv2.cvtColor(x.reshape(32, 32, 3), cv2.COLOR_BGR2GRAY) for x in x_test])

In [7]:
train_hog_features = np.array([hog(x, orientations=8, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=False) for x in train_data_gray])
test_hog_features = np.array([hog(x, orientations=8, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=False) for x in test_data_gray])

In [8]:
pca = PCA(n_components=200, random_state=42)
train_pca_features = pca.fit_transform(train_hog_features)
test_pca_features = pca.transform(test_hog_features)

In [9]:
y_train = y_train.ravel()
y_test = y_test.ravel()

In [10]:
clf = SVC(kernel='rbf')
clf.fit(train_pca_features, y_train)

SVC()

In [11]:
y_pred = clf.predict(test_pca_features)

In [12]:
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred, average='weighted')
rec = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print('Accuracy:', acc)
print('Precision:', prec)
print('Recall:', rec)
print('F1-Score:', f1)

Accuracy: 0.626
Precision: 0.6249679364491738
Recall: 0.626
F1-Score: 0.6246989635968799


### 4. Використання методу SIFT для витягування ознак з зображень

In [24]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [34]:
x_train_gray = np.array([cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) for img in x_train])
x_test_gray = np.array([cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) for img in x_test])

In [35]:
sift = cv2.SIFT_create()

In [36]:
sift_train_features = []
sift_train_labels = []
for i in range(len(x_train_gray)):
    keypoints, descriptors = sift.detectAndCompute(x_train_gray[i], None)
    if descriptors is not None:
        sift_train_features.append(descriptors)
        sift_train_labels.append(y_train[i])

In [37]:
sift_test_features = []
sift_test_labels = []
for i in range(len(x_test_gray)):
    keypoints, descriptors = sift.detectAndCompute(x_test_gray[i], None)
    if descriptors is not None:
        sift_test_features.append(descriptors)
        sift_test_labels.append(y_test[i])

In [38]:
max_descriptors = max([len(desc) for desc in sift_train_features])
sift_train_features_arr = np.zeros((len(sift_train_features), max_descriptors, 128))
for i in range(len(sift_train_features)):
    for j in range(len(sift_train_features[i])):
        sift_train_features_arr[i][j] = sift_train_features[i][j]

In [39]:
max_descriptors = max([len(desc) for desc in sift_test_features])
sift_test_features_arr = np.zeros((len(sift_test_features), max_descriptors, 128))
for i in range(len(sift_test_features)):
    for j in range(len(sift_test_features[i])):
        sift_test_features_arr[i][j] = sift_test_features[i][j]

In [40]:
new_sift_train_features_arr = sift_train_features_arr.reshape(len(sift_train_features_arr), -1)
new_sift_test_features_arr = sift_test_features_arr.reshape(len(sift_test_features_arr), -1)

In [41]:
pca = PCA(n_components=200)
train_pca_features_shift = pca.fit_transform(new_sift_train_features_arr)
test_pca_features_shift = pca.fit_transform(new_sift_test_features_arr)

In [42]:
new_sift_train_labels = []
for i in sift_train_labels:
  new_sift_train_labels.append(i[0])

In [43]:
new_sift_test_labels = []
for i in sift_test_labels:
  new_sift_test_labels.append(i[0])

In [44]:
clf = SVC(kernel='rbf')
clf.fit(train_pca_features_shift, new_sift_train_labels)

SVC()

In [45]:
y_pred = clf.predict(test_pca_features_shift)

In [46]:
accuracy = accuracy_score(new_sift_test_labels, y_pred)
precision = precision_score(new_sift_test_labels, y_pred, average='macro')
recall = recall_score(new_sift_test_labels, y_pred, average='macro')
f1 = f1_score(new_sift_test_labels, y_pred, average='macro')


print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Accuracy: 0.19015637530072174
Precision: 0.18680431447268966
Recall: 0.1902400194488252
F1-score: 0.1872264902408443


## Висновок

**При використанні методу HOG для витягування ознак з зображень точніть вийшла 0.626 а з методом SIFT 0.190. 
Отже, для даного датасету метод HOG працює дещо краще.**